In [1]:
import tensorflow as tf
import tensorflow.keras as kr
import nibabel as nib
import numpy as np
import os
from glob import glob
import matplotlib.pylab as plt
import pydicom as dicom
import multiprocessing as mp

folder_path = [os.path.join(r"/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data", i) for i in os.listdir(r"/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data")]
imgs = []

for i in folder_path:
    imgs.append(glob(i+"/*"))
print("Done")
folder_path = folder_path[1:]
imgs = imgs[1:]
print(imgs[0])
outputpath = []
inputpath = []
for j in imgs:
    for i in j:
        if "seg.nii.gz" in i.split("_"):
            outputpath.append(i)
for j in imgs:
    temppath = []
    for i in j:
        if "seg.nii.gz" not in i.split("_"):
            temppath.append(i)
    inputpath.append(temppath)
    
#load data
inputimg = np.load('input_dataset.npy')
outputimg = np.load('output_dataset.npy')





inputimg = np.array(inputimg, dtype=np.float32)
outputimg = np.array(outputimg, dtype=np.float32)
outputimg = np.expand_dims(outputimg, axis=3)




x_train = inputimg[:1126].copy()
y_train = outputimg[:1126].copy()
x_train.shape[0] == y_train.shape[0]
x_eval = inputimg[1126:1189].copy()
y_eval = outputimg[1126:1189].copy()
print(x_eval.shape, y_eval.shape)
print(x_eval.shape[0] == y_eval.shape[0])
x_test = inputimg[1189:].copy()
y_test = outputimg[1189:].copy()

print(y_train.shape,y_test.shape,y_eval.shape)
print(x_test.shape, y_test.shape)
print(x_test.shape[0] == y_test.shape[0])

#shuffle data
np.random.seed(0)

np.random.shuffle(x_train)
np.random.shuffle(y_train)

np.random.shuffle(x_eval)
np.random.shuffle(y_eval)

np.random.shuffle(x_test)
np.random.shuffle(y_test)

print(tf.config.list_physical_devices('GPU'))
print()
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


2024-06-24 20:59:40.838900: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-24 20:59:41.250170: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-24 20:59:42.290892: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Done
['/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_flair.nii.gz', '/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_seg.nii.gz', '/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_t1.nii.gz', '/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_t1ce.nii.gz', '/mnt/d/Fabio/deeplearningdata/archive_1/BraTS2021_Training_Data/BraTS2021_00000/BraTS2021_00000_t2.nii.gz']
(63, 240, 240, 4) (63, 240, 240, 1)
True
(1126, 240, 240, 1) (62, 240, 240, 1) (63, 240, 240, 1)
(62, 240, 240, 4) (62, 240, 240, 1)
True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12333498110601613583
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5833228288
locality {
  bus_id: 1
  links {
  }
}

2024-06-24 20:59:55.839482: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-24 20:59:56.114017: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-24 20:59:56.114050: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-24 20:59:56.122967: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-24 20:59:56.123007: I external/local_xla/xla/stream_executor

In [2]:
from tensorflow.keras import layers
inputs = kr.Input((240,240,4))
x = layers.Conv2D(64, 3, strides=2, activation="relu", padding="same")(inputs)
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
x = layers.Conv2D(128, 3, strides=2, activation="relu", padding="same")(x)
x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
x = layers.Conv2D(256, 3, strides=2, padding="same", activation="relu")(x)
x = layers.Conv2D(256, 3, activation="relu", padding="same")(x)

x = layers.Conv2DTranspose(256, 3, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(256, 3, activation="relu", padding="same", strides=2)(x)
x = layers.Conv2DTranspose(128, 3, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(128, 3, activation="relu", padding="same", strides=2)(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", padding="same", strides=2)(x)

outputs = layers.Conv2D(4, 3, activation="softmax", padding="same")(x)
model = kr.Model(inputs,outputs)
model.summary()


2024-06-24 20:59:56.621407: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-24 20:59:56.621557: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-24 20:59:56.621606: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-24 20:59:56.621863: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-24 20:59:56.621881: I tensorflow/core/common_runtime/gpu/gpu

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 240, 240, 4)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 120, 120, 64)   │         2,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 120, 120, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 60, 60, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 30, 30, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 30, 30, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 30, 30, 256)    │       590,080 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 60, 60, 256)    │       590,080 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 60, 60, 128)    │       295,040 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_3              │ (None, 120, 120, 128)  │       147,584 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_4              │ (None, 120, 120, 64)   │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_5              │ (None, 240, 240, 64)   │        36,928 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 240, 240, 4)    │         2,308 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881,796 (10.99 MB)

 Trainable params: 2,881,796 (10.99 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:

model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy",metrics=['accuracy'])
callbacks = [
    kr.callbacks.ModelCheckpoint("test.keras",save_best_only=True)
 ]
print("good")

good


: 

In [4]:

model.fit(x_train, y_train,
epochs=100,
batch_size=32,)

In [ ]:
print(x_train.shape)
print(y_train.shape)

(1126, 240, 240, 4)
(1126, 240, 240, 1)


In [ ]:
y = np.random.random((240,240,4))
y = y.reshape(-1, 240, 240, 4)
l = model.predict(y)
l.shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


(1, 240, 240, 4)

array([[[[0.9939632 , 0.06212173, 0.68608622, 0.04782734],
         [0.23835742, 0.64834117, 0.04466015, 0.21668385],
         [0.55677694, 0.03776949, 0.43086893, 0.03729681],
         ...,
         [0.80622003, 0.44811012, 0.4253279 , 0.88438223],
         [0.79741461, 0.19310302, 0.16764181, 0.52280887],
         [0.61997443, 0.62161204, 0.01877182, 0.73815264]],

        [[0.46483469, 0.29863955, 0.37491456, 0.78061745],
         [0.10566782, 0.14828986, 0.49191619, 0.61223187],
         [0.60824256, 0.23063928, 0.92257642, 0.97010094],
         ...,
         [0.29118969, 0.61306344, 0.42013154, 0.38675303],
         [0.97228577, 0.17467227, 0.19592567, 0.37797332],
         [0.45902055, 0.59779893, 0.90963823, 0.5504914 ]],

        [[0.30663427, 0.34698519, 0.81150578, 0.48226193],
         [0.91573099, 0.58087374, 0.56426583, 0.34699964],
         [0.98805063, 0.29629884, 0.66394246, 0.21410737],
         ...,
         [0.50496119, 0.08933586, 0.91958637, 0.48580371],
         [